This script was an initial exploration of using generators to fit semantic spaces from disk (lower memory foot print). It's hard to work with conceptually unfortunately

In [1]:
import time
start_time = time.time()

import json

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import numpy as np

import sys
sys.path.append("../libraries/")
from selector import split_data
from semanticClassifiers import docTopTransformer

In [2]:
# convert input to bag of words

def preprocess_line(this_line):
    """
    Input  : docuemnt list
    Purpose: preprocess text (tokenize, removing stopwords, and stemming)
    Output : preprocessed text
    """
    # initialize regex tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    # create English stop words list
    en_stop = set(stopwords.words('english'))
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    # list for tokenized documents in loop
    texts = []
    
    # clean and tokenize document string
    if this_line:
        raw = this_line.lower()
        tokens = tokenizer.tokenize(raw)
        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        # add tokens to list
        return stemmed_tokens
    else:
        return []

def clean_text_gen(path, field):
    import json
    
    with open(path, 'r') as fileHandle:
        for line in fileHandle:
            this_entry = json.loads(line)
            yield preprocess_line(this_entry.get(field))
        

In [5]:
import time

# convert to bag of words
from gensim.corpora import Dictionary
import gensim.matutils as matutils

field = 'reviewText'
path = '../data/Sports_and_Outdoors_Reviews_training_new.json'
text_gen = clean_text_gen(path, field)

start_time = time.time()
# train a document-topic model        
this_dict = Dictionary(text_gen)
end_time = time.time()
print(end_time - start_time)

945.5390586853027
